### Import libary

In [1]:
from sympy import *

### Add symbols variables

Constant

In [3]:
mass_rod_1, mass_rod_2, inertia_rod_1, inertia_rod_2, len_rod_1, len_rod_2, g = symbols('m_1 m_2 I_1 I_2 l_1 l_2 g', positive=True)

Generalized coordinates

In [4]:
t = symbols('t', positive = True)
tt1, tt2 = symbols('theta_1 theta_2', cls=Function)
d_tt1 = tt1(t).diff(t)
d_tt2 = tt2(t).diff(t)

##  Forward kinematics

In [5]:
#point_1 = chrono.ChVectorD(len_rod_1/2*np.sin(tt1), -len_rod_1/2*np.cos(tt2),0.)
#point_2 = point_1*2. + chrono.ChVectorD(len_rod_2/2*np.sin(tt1+tt2), - len_rod_2/2*np.cos(tt1+tt2),0.)
#print(point_2)

point_1 = Matrix([[len_rod_1/2*sin(tt1(t))],[-len_rod_1/2*cos(tt2(t))]])
point_2 = 2*point_1 + Matrix([[len_rod_2/2*sin(tt1(t)+tt2(t))],[- len_rod_2/2*cos(tt1(t)+tt2(t))]])
print(point_2)

Matrix([[l_1*sin(theta_1(t)) + l_2*sin(theta_1(t) + theta_2(t))/2], [-l_1*cos(theta_2(t)) - l_2*cos(theta_1(t) + theta_2(t))/2]])


## Equation of motion - SymPy


1. Potential energy

In [6]:
EngP = (-mass_rod_1*g*len_rod_1/2*cos(tt1(t)) 
        - mass_rod_2*g*(len_rod_2/2*cos(tt1(t) + tt2(t)) + len_rod_1*cos(tt1(t))))


2. Kinetic energy

In [7]:

EngT_1 = 1/2 * inertia_rod_1 * d_tt1 ** 2

EngT_2 = (1/2*(mass_rod_2*len_rod_1**2 + inertia_rod_2 
        + 2*mass_rod_2*len_rod_1*len_rod_2/2*cos(tt2(t)))*d_tt1**2 + 1/2*inertia_rod_2*d_tt2**2
        + (inertia_rod_2 + mass_rod_2*len_rod_1*len_rod_2/2*cos(tt2(t)))*d_tt1*d_tt2)

EngT = EngT_1 + EngT_2

Euler–Lagrange equation

In [9]:
L = EngT - EngP
u = symbols('tau', cls=Function)
EoM_1 = Eq(L.diff(d_tt1).diff(t) - L.diff(tt1(t)),0)
EoM_2 = Eq(L.diff(d_tt2).diff(t) - L.diff(tt2(t)),u(t))
print_latex(EoM_1)

1.0 I_{1} \frac{d^{2}}{d t^{2}} \theta_{1}{\left(t \right)} + \frac{g l_{1} m_{1} \sin{\left(\theta_{1}{\left(t \right)} \right)}}{2} - g m_{2} \left(- l_{1} \sin{\left(\theta_{1}{\left(t \right)} \right)} - \frac{l_{2} \sin{\left(\theta_{1}{\left(t \right)} + \theta_{2}{\left(t \right)} \right)}}{2}\right) - 1.0 l_{1} l_{2} m_{2} \sin{\left(\theta_{2}{\left(t \right)} \right)} \frac{d}{d t} \theta_{1}{\left(t \right)} \frac{d}{d t} \theta_{2}{\left(t \right)} - \frac{l_{1} l_{2} m_{2} \sin{\left(\theta_{2}{\left(t \right)} \right)} \left(\frac{d}{d t} \theta_{2}{\left(t \right)}\right)^{2}}{2} + \left(I_{2} + \frac{l_{1} l_{2} m_{2} \cos{\left(\theta_{2}{\left(t \right)} \right)}}{2}\right) \frac{d^{2}}{d t^{2}} \theta_{2}{\left(t \right)} + \left(I_{2} + l_{1}^{2} m_{2} + l_{1} l_{2} m_{2} \cos{\left(\theta_{2}{\left(t \right)} \right)}\right) \frac{d^{2}}{d t^{2}} \theta_{1}{\left(t \right)} = 0


### Manipulator equation

In [19]:
M = Matrix([[inertia_rod_1 + inertia_rod_2 + mass_rod_2*len_rod_1**2 + 2*mass_rod_2*len_rod_1*len_rod_2/2*cos(tt2(t)),
          inertia_rod_2 + mass_rod_2*len_rod_1*len_rod_2/2*cos(tt2(t))],
         [inertia_rod_2 + mass_rod_2*len_rod_1*len_rod_2/2*cos(tt2(t)), inertia_rod_2]])

C = Matrix([[-2*mass_rod_2*len_rod_1*len_rod_2/2*sin(tt2(t))*d_tt2, -mass_rod_2*len_rod_1*len_rod_2/2*sin(tt2(t))*d_tt2],
         [mass_rod_2*len_rod_1*len_rod_2/2*sin(tt2(t))*d_tt2, 0]])

G = Matrix([[-mass_rod_1*g*len_rod_1/2*sin(tt1(t)) - mass_rod_2*g*(len_rod_1*sin(tt1(t))+ len_rod_2/2*sin(tt1(t) + tt2(t)))],
                    [-mass_rod_2*g*len_rod_2/2*sin(tt1(t)+tt2(t))]])

B = Matrix([[0],[1]])

dq_G = Matrix([[g*(mass_rod_1*len_rod_1/2 + mass_rod_2*len_rod_1 + mass_rod_2*len_rod_2/2), mass_rod_2*len_rod_2/2 * g],
                   [mass_rod_2*len_rod_2/2 * g, mass_rod_2*len_rod_2/2 * g]])


In [28]:
print_latex()

\left[\begin{matrix}- \frac{g l_{1} m_{1}}{2} - g m_{2} \left(l_{1} + \frac{l_{2}}{2}\right)\\- \frac{g l_{2} m_{2}}{2}\end{matrix}\right]


In [ ]:

#Alin = Matrix([[zeros((2,2)),eye(2)],[inv(M) @ dq_G, np.zeros((2,2))]])
#Blin = np.block([[np.zeros((2,1))],[la.inv(listM) @ listB]])

#Q = np.diag((10.,10.,1.,1.))
#R = np.asarray([1])

#X, K, E = lqr(Alin,Blin,Q,R)

# Equation of motion - numpy version

Libaries

In [1]:
from cmath import cos
import pychrono.core as chrono
import pychrono.irrlicht as chrirr
import numpy as np
import numpy.linalg as la
import scipy.linalg as cla
import matplotlib.pyplot as plt

Function for calculate coefficients controller

In [2]:
def lqr(A,B,Q,R):
    X = np.matrix(cla.solve_continuous_are(A, B, Q, R))
    K = np.matrix(1/R*(B.T*X))
    E= cla.eig(A-B*K)[0]
    return X, K, E  

Parameters

In [3]:
xz_rod = 0.05
len_rod_1 = 0.3
len_rod_2 = 0.6
density = 1000
g = 9.81

mass_rod_1 = xz_rod**2*len_rod_1*density
mass_rod_2 = xz_rod**2*len_rod_2*density

inertia_rod_1 = len_rod_1**2*mass_rod_1
inertia_rod_2 = len_rod_2**2*mass_rod_2

The equlibrium state

In [4]:
tt1 = np.pi
tt2 = 0
d_tt1 = 0
d_tt2 = 0

Manipulator equation

In [5]:

listM = np.asarray([[inertia_rod_1 + inertia_rod_2 + mass_rod_2*len_rod_1**2 + 2*mass_rod_2*len_rod_1*len_rod_2/2*np.cos(tt2),
          inertia_rod_2 + mass_rod_2*len_rod_1*len_rod_2/2*np.cos(tt2)],
         [inertia_rod_2 + mass_rod_2*len_rod_1*len_rod_2/2*np.cos(tt2), inertia_rod_2]])


listC = np.asarray([[-2*mass_rod_2*len_rod_1*len_rod_2/2*np.sin(tt2)*d_tt2, -mass_rod_2*len_rod_1*len_rod_2/2*np.sin(tt2)*d_tt2],
         [mass_rod_2*len_rod_1*len_rod_2/2*np.sin(tt2)*d_tt2, 0]])

listG = np.asarray([[-mass_rod_1*g*len_rod_1/2*np.sin(tt1) - mass_rod_2*g*(len_rod_1*np.sin(tt1)+ len_rod_2/2*np.sin(tt1 + tt2))],
                    [-mass_rod_2*g*len_rod_2/2*np.sin(tt1+tt2)]])


listB = np.asarray([[0],[1]])

dq_G = np.asarray([[g*(mass_rod_1*len_rod_1/2 + mass_rod_2*len_rod_1 + mass_rod_2*len_rod_2/2), mass_rod_2*len_rod_2/2 * g],
                   [mass_rod_2*len_rod_2/2 * g, mass_rod_2*len_rod_2/2 * g]])

[-15.27959562+0.j          -3.67987523+0.j
  -2.86285367+0.16265233j  -2.86285367-0.16265233j]


Linearization

In [ ]:

Alin = np.block([[np.zeros((2,2)),np.eye(2)],[la.inv(listM) @ dq_G, np.zeros((2,2))]])
Blin = np.block([[np.zeros((2,1))],[la.inv(listM) @ listB]])

Q = np.diag((10.,10.,1.,1.))
R = np.asarray([1])

X, K, E = lqr(Alin,Blin,Q,R)


Coefficents controller

In [6]:

print(K)

[[-95.33425181 -44.15974824 -29.72448497 -17.59465727]]
